In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from statsmodels.graphics.tsaplots import month_plot, quarter_plot

In [11]:
sales_df = pd.read_csv("../data/daily_revenue.csv", 
                       parse_dates=["date"], 
                       index_col=["date"], 
                       dayfirst = True)

In [12]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1795 entries, 2018-01-01 to 2022-11-30
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   revenue        1795 non-null   object
 1   discount_rate  1795 non-null   object
 2   coupon_rate    1795 non-null   object
dtypes: object(3)
memory usage: 56.1+ KB


In [13]:
sales_df.head()

,revenue,discount_rate,coupon_rate
date,,,
2018-01-01,"6,270,839",34.27%,1.09%
2018-01-02,"8,922,076",30.87%,1.08%
2018-01-03,"8,446,101",28.11%,1.01%
2018-01-04,"7,785,798",27.32%,0.96%
2018-01-05,"6,375,303",25.70%,0.90%


In [21]:
sales_df["revenue"] = sales_df["revenue"].str.replace(",", "").astype(float)
sales_df["discount_rate"] = sales_df["discount_rate"].str.replace("%", "").astype(float)
sales_df["coupon_rate"] = sales_df["coupon_rate"].str.replace("%", "").astype(float)

In [22]:
sales_df

,revenue,discount_rate,coupon_rate
date,,,
2018-01-01,6270839.0,34.27,1.09
2018-01-02,8922076.0,30.87,1.08
2018-01-03,8446101.0,28.11,1.01
2018-01-04,7785798.0,27.32,0.96
2018-01-05,6375303.0,25.70,0.90
...,...,...,...
2022-11-26,26934947.0,30.41,0.38
2022-11-27,31890868.0,29.75,0.35
2022-11-28,32100918.0,25.27,8.93


In [23]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1795 entries, 2018-01-01 to 2022-11-30
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   revenue        1795 non-null   float64
 1   discount_rate  1795 non-null   float64
 2   coupon_rate    1795 non-null   float64
dtypes: float64(3)
memory usage: 56.1 KB


np.float64(21332122978.0)